In [ ]:
# ANOVA Benjamini-Hochberg
#Change the NAME.csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
# Import multipletests from the correct module
from statsmodels.stats.multitest import multipletests

# Load the data
try:
    df = pd.read_csv('NAME.csv')
except FileNotFoundError:
    print("NAME.csv not found. Please upload the file.")
    # Create a dummy file for demonstration if the file is not found
    data = {'Classe': ['Class1', 'Class1', 'Class1', 'Class2', 'Class2', 'Class2', 'Class3', 'Class3', 'Class3']}
    for i in range(1, 101):
        data[f'{i}'] = np.random.rand(9) + np.random.randint(0, 3, 9)
    df = pd.DataFrame(data)
    df.to_csv('NAME.csv', index=False)
    df = pd.read_csv('NAME.csv')


# Assume the first column is 'Class' and the rest are wavenumbers
# Corrected 'Classe' to 'Class' based on the KeyError and global variables
classes = df['Class']
wavenumbers = df.columns[1:].astype(float)
spectra = df.iloc[:, 1:]

# Calculate mean and standard deviation for each class
class_means = spectra.groupby(classes).mean()
class_stds = spectra.groupby(classes).std()

# Get the class names
class_names = class_means.index

# Calculate the difference in means between groups (example: Class1 - Class2)
# You can adjust this to compare different pairs of classes
if len(class_names) >= 2:
    mean_diff = class_means.loc[class_names[0]] - class_means.loc[class_names[1]]
    std_sum_sq = class_stds.loc[class_names[0]]**2 + class_stds.loc[class_names[1]]**2
    std_diff_approx = np.sqrt(std_sum_sq) # Approximate standard deviation of the difference

    # Plot the difference in mean with standard deviation bands
    plt.figure(figsize=(12, 6))
    plt.plot(wavenumbers, mean_diff, label=f'Mean Difference ({class_names[0]} - {class_names[1]})')
    plt.fill_between(wavenumbers, mean_diff - std_diff_approx, mean_diff + std_diff_approx, color='gray', alpha=0.3, label='Approx. Std Dev Band')
    plt.xlabel('Wavenumber')
    plt.ylabel('Mean Difference')
    plt.title(f'Mean Difference Spectrum ({class_names[0]} vs {class_names[1]}) with Approximate Std Dev Band')
    plt.legend()
    plt.grid(True)
    plt.gca().invert_xaxis() # Invert x-axis for wavenumber plots
    plt.show()

# Perform point-by-point ANOVA
anova_p_values = []
for col in spectra.columns:
    groups = [spectra[col][classes == cls] for cls in class_names]
    # Handle potential empty groups or groups with only one sample which can cause NaNs in ANOVA
    # Filter out groups with less than 2 samples
    valid_groups = [g for g in groups if len(g) >= 2]
    if len(valid_groups) >= 2:
        f_statistic, p_value = stats.f_oneway(*valid_groups)
        anova_p_values.append(p_value)
    else:
        # If not enough valid groups for ANOVA, append NaN or another indicator
        anova_p_values.append(np.nan)


anova_p_values = np.array(anova_p_values)

# Apply Benjamini-Hochberg correction for multiple testing
# Use the directly imported multipletests function
# Filter out NaN values before correction
valid_p_values = anova_p_values[~np.isnan(anova_p_values)]
if len(valid_p_values) > 0:
    reject, pvals_corrected_valid, _, _ = multipletests(valid_p_values, method='fdr_bh') # fdr_bh is Benjamini-Hochberg
    # Create a full p-value array with NaNs in the original positions
    pvals_corrected = np.full(anova_p_values.shape, np.nan)
    pvals_corrected[~np.isnan(anova_p_values)] = pvals_corrected_valid
else:
    pvals_corrected = np.full(anova_p_values.shape, np.nan)


# Plot corrected p-values
plt.figure(figsize=(12, 6))
# Plot only if there are valid corrected p-values
if np.any(~np.isnan(pvals_corrected)):
    plt.plot(wavenumbers, pvals_corrected, label='Benjamini-Hochberg Corrected p-values')
    plt.axhline(y=0.05, color='r', linestyle='--', label='Significance Level (0.05)')
    plt.xlabel('Wavenumber')
    plt.ylabel('Corrected p-value')
    plt.title('Point-by-Point ANOVA Corrected p-values (Benjamini-Hochberg)')
    plt.legend()
    plt.grid(True)
    plt.gca().invert_xaxis() # Invert x-axis for wavenumber plots
    plt.show()
else:
    print("No valid p-values to plot after ANOVA and correction.")

In [ ]:
# Mean difference analysis for other classes

import matplotlib.pyplot as plt
import numpy as np
# Function to plot mean difference and standard deviation band for two classes
def plot_mean_difference(class1_name, class2_name, class_means, class_stds, wavenumbers):
    if class1_name in class_means.index and class2_name in class_means.index:
        mean_diff = class_means.loc[class1_name] - class_means.loc[class2_name]
        std_sum_sq = class_stds.loc[class1_name]**2 + class_stds.loc[class2_name]**2
        std_diff_approx = np.sqrt(std_sum_sq)

        plt.figure(figsize=(12, 6))
        plt.plot(wavenumbers, mean_diff, label=f'Mean Difference ({class1_name} - {class2_name})')
        plt.fill_between(wavenumbers, mean_diff - std_diff_approx, mean_diff + std_diff_approx, color='gray', alpha=0.3, label='Approx. Std Dev Band')
        plt.xlabel('Wavenumber')
        plt.ylabel('Mean Difference')
        plt.title(f'Mean Difference Spectrum ({class1_name} vs {class2_name}) with Approximate Std Dev Band')
        plt.legend()
        plt.grid(True)
        plt.gca().invert_xaxis()
        plt.show()
    else:
        print(f"One or both classes '{class1_name}' or '{class2_name}' not found in the data.")

# Iterate through all unique pairs of classes and plot the mean difference
from itertools import combinations

for class1, class2 in combinations(class_names, 2):
    plot_mean_difference(class1, class2, class_means, class_stds, wavenumbers)

In [ ]:
# Benjamini-Hochberg analysis for other classes

import matplotlib.pyplot as plt
import numpy as np
# Perform point-by-point ANOVA for all pairs of classes and plot corrected p-values

# Get all unique class pairs
class_pairs = list(combinations(class_names, 2))

for class1, class2 in class_pairs:
    print(f"\nPerforming ANOVA and plotting for {class1} vs {class2}")

    # Filter data for the two classes
    df_subset = df[df['Class'].isin([class1, class2])]
    classes_subset = df_subset['Class']
    spectra_subset = df_subset.iloc[:, 1:]

    # Perform point-by-point ANOVA for the subset
    anova_p_values_subset = []
    for col in spectra_subset.columns:
        group1 = spectra_subset[col][classes_subset == class1]
        group2 = spectra_subset[col][classes_subset == class2]

        # Ensure groups have at least two samples
        if len(group1) >= 2 and len(group2) >= 2:
            f_statistic, p_value = stats.f_oneway(group1, group2)
            anova_p_values_subset.append(p_value)
        else:
            # Not enough data for ANOVA for this wavenumber and pair
            anova_p_values_subset.append(np.nan)

    anova_p_values_subset = np.array(anova_p_values_subset)

    # Apply Benjamini-Hochberg correction for multiple testing
    valid_p_values_subset = anova_p_values_subset[~np.isnan(anova_p_values_subset)]
    if len(valid_p_values_subset) > 0:
        reject_subset, pvals_corrected_subset_valid, _, _ = multipletests(valid_p_values_subset, method='fdr_bh')
        pvals_corrected_subset = np.full(anova_p_values_subset.shape, np.nan)
        pvals_corrected_subset[~np.isnan(anova_p_values_subset)] = pvals_corrected_subset_valid
    else:
        pvals_corrected_subset = np.full(anova_p_values_subset.shape, np.nan)

    # Plot corrected p-values for the pair
    plt.figure(figsize=(12, 6))
    if np.any(~np.isnan(pvals_corrected_subset)):
        plt.plot(wavenumbers, pvals_corrected_subset, label='Benjamini-Hochberg Corrected p-values')
        plt.axhline(y=0.05, color='r', linestyle='--', label='Significance Level (0.05)')
        plt.xlabel('Wavenumber')
        plt.ylabel('Corrected p-value')
        plt.title(f'Point-by-Point ANOVA Corrected p-values ({class1} vs {class2}) (Benjamini-Hochberg)')
        plt.legend()
        plt.grid(True)
        plt.gca().invert_xaxis()
        plt.show()
    else:
        print(f"No valid p-values to plot for {class1} vs {class2} after ANOVA and correction.")
